# Hawk-Dove-Bourgeois Game

In this notebook, we'll consider the situation where bourgeoises emerge as a new power in Hawk-Dove game.

Bourgeoises behave as follows.

- If they own the land that the food V exists, they behave as a hawk.
- If not, they behave as a dove.
- The probability of owning the land is p.
- The above is equivalent to behaving as a hawk or a dove with the probabilities p and 1-p, respectively.
- Bourgeoises don't compete with themselves like doves.

For simplicity, we set p to 0.5 this time.  
The expected payoff that a bourgeois gains when competing with a hawk is 
$$ \frac{V-C}{2} \times p + 0 \times (1 - p) = \frac{V-C}{4} $$
Similarly, we can calculate the expected payoff in the other cases and obtain the following payoff matrix.

| | Hawk | Dove | Bourgeois |
| ---- | ---- | ---- | ---- |
| **Hawk** | $$\frac{V-C}{2}$$ | $$V$$ | $$\frac{3V-C}{4}$$ |
| **Dove** | $$0$$ | $$\frac{V}{2}$$ | $$\frac{V}{4}$$ |
| **Bourgeois** | $$\frac{V-C}{4}$$ | $$\frac{3V}{4}$$ | $$\frac{V}{2}$$ |


In [ ]:
# import necessary packages
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
!jupyter nbextension enable --py widgetsnbextension

### Variable Parameters
- V: the amount of food
- C: cost
- dt: update interval
- num_iter: the number of iterations
- init_distr: initial population distribution of groups (Initial share of each pure strategy)
    - (share of hawks, share of doves, share of bourgeoises)

In [ ]:
# key -> index (H = Hawk, D = Dove, B = Bourgeois)
H = 0
D = 1
B = 2

V = 1
C = 2
p = 0.5

# payoff matrix
A = np.array([
    [(V - C) / 2, V, (V - C) / 2 * p + V * (1 - p)],
    [0, V / 2, V / 2 * (1 - p)],
    [(V - C) / 2 * p, V * p + V / 2 * (1 - p), V / 2]
])
print('*** payoff matrix ***')
groups = ['Hawk', 'Dove', 'Bourgeois']
pprint(pd.DataFrame(A, index=groups, columns=groups))

dt = 0.1

num_iter = 1000

# init_distr = (0.5, 0.4, 0.1)  # declare in the next cell

### Run a simulation

If there are no bourgeoises, the share of each group is stable at $$(s_{hawk}, s_{dove}) = (0.5, 0.5)~~~~(V = 1, C = 2)$$  
What would happen if a part of doves were suddenly transformed into bourgeoises, that is, the share changed to $$(s_{hawk}, s_{dove}, s_{bourgeois}) = (0.5, 0.4, 0.1)$$

In [ ]:
init_distr = (0.5, 0.4, 0.1)  # (share of hawks, share of doves, share of bourgeoises)
# assert sum(init_distr) == 1.

s_h, s_d, s_b = init_distr
log = []
for t in tqdm(range(num_iter)):
    # calculate the payoff that each group gains
    u_h = A[H][H] * s_h + A[H][D] * s_d + A[H][B] * s_b
    u_d = A[D][H] * s_h + A[D][D] * s_d + A[D][B] * s_b
    u_b = A[B][H] * s_h + A[B][D] * s_d + A[B][B] * s_b

    # the average payoff
    avg_u = u_h * s_h + u_d * s_d + u_b * s_b
    
    # update according to replicator dynamics
    s_h += s_h * (u_h - avg_u) * dt
    s_d += s_d * (u_d - avg_u) * dt
    s_b += s_b * (u_b - avg_u) * dt

    log.append((s_h, s_d, s_b))

### Plot how each strategy changes
On the default setting, we can see the dominance of a bourgeois strategy.

In [ ]:
s_h, s_d, s_b = map(lambda x: np.array(list(x)), zip(*log))

fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
ax.plot(s_h, label ='share of hawks', color="#E7556A")
ax.plot(s_d, label ='share of doves', color="#7CC28E")
ax.plot(s_b, label ='share of bourgeoises', color="#0080FF")
ax.grid()
ax.legend(loc='upper right')
fig.show()

### Plot how stable each point is
On the default setting, we can see vectors point to the origin.  

In [ ]:
stop = 100
step = 5

log = []
for i in range(0, stop, step):
    for j in range(0, stop - i, step):
        s_h = i / stop
        s_d = j / stop
        s_b = (stop - i - j) / stop
        
        u_h = A[H][H] * s_h + A[H][D] * s_d + A[H][B] * s_b
        u_d = A[D][H] * s_h + A[D][D] * s_d + A[D][B] * s_b
        u_b = A[B][H] * s_h + A[B][D] * s_d + A[B][B] * s_b

        avg_u = u_h * s_h + u_d * s_d + u_b * s_b
        
        d_h = s_h * (u_h - avg_u)
        d_d = s_d * (u_d - avg_u)

        log.append((s_h, s_d, d_h, d_d))
        
s_h, s_d, d_h, d_d = map(list, zip(*log))

fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
ax.quiver(s_h, s_d, d_h, d_d, 
          headwidth=3, headlength=3, headaxislength=3, 
          angles='xy', scale=1.25, scale_units='xy', 
          color="#7CC28E")
ax.set_xlabel('share of hawks', fontsize=14)
ax.set_ylabel('share of doves', rotation=90, fontsize=14)
ax.set_xticks(ticks=np.arange(0, 1.1, 0.1))
ax.set_yticks(ticks=np.arange(0, 1.1, 0.1))
fig.show()

### Extra: Rock Paper Scissors
Since the payoff matrix is cyclic, the share of each strategy is also cyclic.

In [ ]:
R = 0
P = 1
S = 2

A = np.array([
    [0, -1, 1],
    [1, 0, -1],
    [-1, 1, 0]
])
# A = np.array([
#     [0, -1, 1.375],
#     [1, 0, -1],
#     [-1, 1, 0]
# ])
print('*** payoff matrix ***')
moves = ['Rock', 'Paper', 'Scissors']
pprint(pd.DataFrame(A, index=moves, columns=moves))

dt = 0.1

num_iter = 100000

init_distr = (0.3334, 0.3333, 0.3333)
# assert sum(init_distr) == 1.

In [ ]:
s_r, s_p, s_s = init_distr
log = []
for t in tqdm(range(num_iter)):
    u_r = A[R][R] * s_r + A[R][P] * s_p + A[R][S] * s_s
    u_p = A[P][R] * s_r + A[P][P] * s_p + A[P][S] * s_s
    u_s = A[S][R] * s_r + A[S][P] * s_p + A[S][S] * s_s

    avg_u = u_r * s_r + u_p * s_p + u_s * s_s
    
    s_r += s_r * (u_r - avg_u) * dt
    s_p += s_p * (u_p - avg_u) * dt
    s_s += s_s * (u_s - avg_u) * dt
    
    log.append((s_r, s_p, s_s))
    
s_r, s_p, s_s = map(lambda x: np.array(list(x)), zip(*log))

fig, axs = plt.subplots(1, 2, figsize=(10, 5), dpi=100)
for i in range(2):
    axs[i].plot(s_r[:num_iter // 100] if i == 0 else s_r, label ='share of rock', color="#E7556A")
    axs[i].plot(s_p[:num_iter // 100] if i == 0 else s_p, label ='share of paper', color="#7CC28E")
    axs[i].plot(s_s[:num_iter // 100] if i == 0 else s_s, label ='share of scissors', color="#0080FF")
    axs[i].grid()
    axs[i].legend(loc='upper right')
fig.tight_layout()
fig.show()

In [ ]:
# plot vector field
stop = 100
step = 5

log = []
for i in range(0, stop, step):
    for j in range(0, stop - i, step):
        s_r = i / stop
        s_p = j / stop
        s_s = (stop - i - j) / stop
        
        u_r = A[R][R] * s_r + A[R][P] * s_p + A[R][S] * s_s
        u_p = A[P][R] * s_r + A[P][P] * s_p + A[P][S] * s_s
        u_s = A[S][R] * s_r + A[S][P] * s_p + A[S][S] * s_s
        
        avg_u = u_r * s_r + u_p * s_p + u_s * s_s
        
        d_r = s_r * (u_r - avg_u)
        d_p = s_p * (u_p - avg_u)

        log.append((s_r, s_p, d_r, d_p))
        
s_r, s_p, d_r, d_p = map(list, zip(*log))

fig, ax = plt.subplots(figsize=(5, 5), dpi=100)
ax.quiver(s_r, s_p, d_r, d_p, 
          headwidth=3, headlength=3, headaxislength=3, 
          angles='xy', scale=5, scale_units='xy', 
          color="#7CC28E")
ax.set_xlabel('share of rock', fontsize=14)
ax.set_ylabel('share of paper', rotation=90, fontsize=14)
ax.set_xticks(ticks=np.arange(0, 1.1, 0.1))
ax.set_yticks(ticks=np.arange(0, 1.1, 0.1))
fig.show()